In [ ]:
import pandas as pd

#Reading Dataset from CSV file

In [ ]:
captions_path = './survey/survey.csv'
captions = pd.read_csv(captions_path, header=None)
captions.columns = ['captions']

captions_df = pd.DataFrame(captions)
captions_df=captions_df.head(10000)
captions_df['id'] = list(range(5000,6000))[:len(captions_df)]
captions_df.head(10)

,captions,id
0,Sun is shining to the brightest.,5000
1,Annapurna range seen from Pokhara.,5001
2,A girl and a boy are riding bicycle.,5002
3,A deer is eating food in an open area and tige...,5003
4,A girl is crossing the road.,5004
5,A woman is staring out the window.,5005
6,Ugh! Why are you yelling at me?,5006
7,A plastic rewards card lying face down on the ...,5007
8,A boy under the sun.,5008
9,plate full of fruits.,5009


#Using Selenium library with webdriver

In [ ]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from parsel import Selector
import urllib
import urllib.request
import numpy as np
import pandas as pd
import math

In [ ]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [/Users/prekshyabasnet/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


#Initiating Google.com website

In [ ]:
#url = "www.google.com"
driver.get("https://www.google.com")
sleep(1)

#Utility funtion to save the downloaded image and caption in JSON format

In [ ]:
def save_TopFour(src,j,i,caption):
    urllib.request.urlretrieve(src,'./survey/images/%s_%s.jpeg'%(j,i))
    image = {
        "file_name":'%s_%s.jpeg'%(j,i),
        "id": '%s_%s'%(j,i)
    }
    annotation = {
        "caption": caption,
        "id": '%s_%s'%(j,i),
        "image_id": '%s_%s'%(j,i)
    }
    return image, annotation
            

#Webscrapping using Selenium to get four images for each caption

In [ ]:
from tqdm import tqdm 

def get_image_for_captions(captions):
    images = []
    annotations = []
    for j,row in tqdm(captions.iterrows(),desc="Downloading..."):
        sleep(2)
        driver.find_element_by_name('q').send_keys(row['captions'])
        driver.find_element_by_xpath("/html/body/div[2]/div[2]/form/div[2]/div[1]/div[3]/center/input[1]").click()
        
        sleep(2)
        elements = driver.find_elements_by_class_name("q.qs")
        image = [element for element in elements if element.text == "Images"][0]
        image.click()
        imgs = driver.find_elements_by_class_name("rg_i.Q4LuWd")

        for i in range(len(imgs)):
            if(i==4):
                break
            else:
                src  = imgs[i].get_attribute('src')
                alt_txt = imgs[i].get_attribute('alt')
                image,annotation = save_TopFour(src,row['id'],i,row['captions'])
                images.append(image)
                annotations.append(annotation)
        driver.get("https://www.google.com") 
    return images,annotations

#Recursively downloading images until image is found for a caption

In [ ]:
import glob
import re
temp=captions_df
tot_images =[]
tot_annotations=[]
retry = 1
saved_counter=0
while saved_counter!=len(captions_df) and retry<20:
    imgs_saved,annotations_saved = get_image_for_captions(temp)
    tot_images=tot_images+imgs_saved
    tot_annotations = tot_annotations+annotations_saved
    saved_counter = len(tot_images)
    retry = retry+1

    imgs_file = glob.glob('./survey/images/*.jpeg')
    saved = [re.findall("./survey/images/(.*)_\d*.jpeg", file)[0] for file in imgs_file]
    unsaved = [{'id': caption['id'],'captions':caption['captions']} for (index,caption) in captions_df.iterrows() if str(caption['id']) not in saved]
    print(len(unsaved),"----iteration-----",retry)
    unsaved_captions_df = pd.DataFrame(unsaved)  
    temp=unsaved_captions_df

In [ ]:
len(tot_images)==len(tot_annotations), len(tot_images),len(tot_annotations)

(True, 450, 450)

#Saving final JSON with images and annotation

In [ ]:
json_final_200 = {
    "images": tot_images,
    "annotations":tot_annotations
}

In [ ]:
import json
with open('./survey/captions_topN2020_all.json', 'w') as json_file:
    json.dump(json_final_200, json_file)

##After this images are manually selected and the following code is resumed after selection

In [ ]:
import json
# opening the saved json file to preprocess it
with open('./survey/captions_topN2020_all.json') as json_file:
    data = json.load(json_file)

#Get all the selected image caption name to save only images that are selected

In [ ]:
import glob
import re

imgs_file = glob.glob('./survey/images/*.jpeg')
selected_images = [re.findall("./survey/images/(.*).jpeg", file)[0] for file in imgs_file]

In [ ]:
len(selected_images)

127

In [ ]:
annotations = data['annotations']
annotations[0]

{'caption': 'Annapurna range seen from Pokhara. ',
 'id': '5001_0',
 'image_id': '5001_0'}

In [ ]:
images = data['images']
images[0]

{'file_name': '5001_0.jpeg', 'id': '5001_0'}

In [ ]:
list_annotation = []
for image_name in selected_images:
    list_annotation.append([annotate for annotate in annotations if annotate['image_id']==image_name][0])

In [ ]:
list_images = []
for image_name in selected_images:
    list_images.append([image for image in images if image['id']==image_name][0])

In [ ]:
len(list_annotation),len(list_images)

(127, 127)

In [ ]:
list_annotation[0],list_images[0]

({'caption': 'A girl in a white dress at a bus stand.',
  'id': '5021_2',
  'image_id': '5021_2'},
 {'file_name': '5021_2.jpeg', 'id': '5021_2'})

#Construct final selected images and annotatin in following JSON structure and save

In [ ]:
nest = {
        "images":list_images,
        "annotations":list_annotation
    }

In [ ]:
# Saving the file
with open('./survey/captions_topN2020.json', 'w') as json_file:
    json.dump(nest, json_file)